In [92]:
import collections
import serial
import struct
import time


# List Important Registers
Sourced from [datasheet](https://statics.cirrus.com/pubs/proDatasheet/CS5490_F3.pdf)

In [117]:
CMD_READ  = 0x00
CMD_WRITE = 0x40
CMD_PAGE  = 0x80
CMD_INST  = 0xC0

# HARDWARE Page 0
HARDWARE_PAGE = 0 << 8

# Note, page + address are merged into a single 16 bit value
# List Addresses
CONFIG0 = 0x00 + HARDWARE_PAGE
CONFIG1 = 0x01 + HARDWARE_PAGE
MASK = 0x03 + HARDWARE_PAGE
PC = 0x05 + HARDWARE_PAGE
SERIALCTRL = 0x07 + HARDWARE_PAGE
PULSEWIDTH = 0x08 + HARDWARE_PAGE
PULSECTRL = 0x09 + HARDWARE_PAGE
STATUS0 = 0x17 + HARDWARE_PAGE
STATUS1 = 0x18 + HARDWARE_PAGE
STATUS2 = 0x19 + HARDWARE_PAGE
REGLOCK = 0x22 + HARDWARE_PAGE
V_PEAK = 0x24 + HARDWARE_PAGE
I_PEAK = 0x25 + HARDWARE_PAGE
PSDC = 0x30 + HARDWARE_PAGE
ZXNUM = 0x37 + HARDWARE_PAGE

# SOFTWARE Page 16
SOFTWARE_P16 = 16 << 8

# List Addresses
CONFIG2 = 0x00 + SOFTWARE_P16
REGCHK = 0x01 + SOFTWARE_P16
V_INST = 0x02 + SOFTWARE_P16
I_INST = 0x03 + SOFTWARE_P16
P_INST = 0x04 + SOFTWARE_P16
P_AVG = 0x05 + SOFTWARE_P16
I_RMS = 0x06 + SOFTWARE_P16
V_RMS = 0x07 + SOFTWARE_P16
Q_AVG = 0x0E + SOFTWARE_P16
Q_INST = 0x0F + SOFTWARE_P16
S_INST = 0x14 + SOFTWARE_P16
PF = 0x15 + SOFTWARE_P16
TEMP = 0x1B + SOFTWARE_P16  # S Q7.16
P_SUM = 0x1D + SOFTWARE_P16
S_SUM = 0x1E + SOFTWARE_P16
Q_SUM = 0x1F + SOFTWARE_P16
I_DCOFF = 0x20 + SOFTWARE_P16
I_GAIN = 0x21 + SOFTWARE_P16  # U Q2.22
V_DCOFF = 0x22 + SOFTWARE_P16
V_GAIN = 0x23 + SOFTWARE_P16  # U Q2.22
P_OFF = 0x24 + SOFTWARE_P16
I_ACOFF = 0x25 + SOFTWARE_P16
EPSILON = 0x31 + SOFTWARE_P16
SAMPLECOUNT = 0x33 + SOFTWARE_P16
T_GAIN = 0x36 + SOFTWARE_P16  # U Q8.16
T_OFF = 0x37 + SOFTWARE_P16
T_SETTLE = 0x39 + SOFTWARE_P16
LOAD_MIN = 0x40 + SOFTWARE_P16
SYS_GAIN = 0x3C + SOFTWARE_P16
SYS_TIME = 0x3D + SOFTWARE_P16

# SOFTWARE Page 17
SOFTWARE_P17 = 17 << 8

# List Addresses
VSAG_DUR = 0x00 + SOFTWARE_P17
VSAG_LEVEL = 0x01 + SOFTWARE_P17
IOVER_DUR = 0x04 + SOFTWARE_P17
IOVER_LEVEL = 0x05 + SOFTWARE_P17

# SOFTWARE Page 18
SOFTWARE_P18 = 18 << 8

# List Addresses
IZX_LEVEL = 0x18 + SOFTWARE_P18
PULSERATE = 0x1C + SOFTWARE_P18
INT_GAIN = 0x2B + SOFTWARE_P18
VSWELL_DUR = 0x2E + SOFTWARE_P18
VSWELL_LEVEL = 0x2F + SOFTWARE_P18
VZX_LEVEL = 0x3A + SOFTWARE_P18
CYCLECOUNT = 0x3E + SOFTWARE_P18
SCALE = 0x3F + SOFTWARE_P18

SIGNED = True
UNSIGNED = False

# Start of register metadata, will help with register dumps, value conversions, sanity check
regtable = (  # (address, string:name, int:q, bool:SIGNED, int:default)
    # HW page 0
    (CONFIG0, 'config0', 0, UNSIGNED, 0xC02000),
    (CONFIG1, 'config1', 0, UNSIGNED, 0x00EEEE),
    (SERIALCTRL, 'serialctrl', 0, UNSIGNED, 0x02004D),
    (STATUS0, 'status0', 0, UNSIGNED, 0x800000),
    (STATUS1, 'status1', 0, UNSIGNED, 0x801800),
    (STATUS2, 'status2', 0, UNSIGNED, 0),
    (V_PEAK, 'v_peak', 23, SIGNED, 0),
    (I_PEAK, 'i_peak', 23, SIGNED, 0),
    
    # SW page 16
    (CONFIG2, 'config2', 0, UNSIGNED, 0x100200),
    (REGCHK, 'regchk', 0, UNSIGNED, 0),
    (V_INST, 'v_inst', 23, SIGNED, 0),
    (I_INST, 'i_inst', 23, SIGNED, 0),
    (P_INST, 'p_inst', 23, SIGNED, 0),
    (P_AVG, 'p_avg', 23, SIGNED, 0),
    (I_RMS, 'i_rms', 24, UNSIGNED, 0),
    (V_RMS, 'v_rms', 24, UNSIGNED, 0),
    (Q_AVG, 'q_avg', 23, SIGNED, 0),

    (TEMP, 'temp', 16, SIGNED, 0),
    (I_GAIN, 'i_gain', 22, UNSIGNED, 0x400000),
    (V_GAIN, 'v_gain', 22, UNSIGNED, 0x400000),
    (T_GAIN, 't_gain', 16, UNSIGNED, 0x06B716),
)

reginfo = collections.OrderedDict(((r[0], r[1:]) for r in regtable))

# Instructions (use with CMD_INST)

# Controls
SOFT_RESET = 0x1
STANDBY = 0x2
WAKEUP = 0x3
SINGLE_CONV =  0x14
CONT_CONV = 0x15
HALT_CONV = 0x18

# Calibration
I_AC_CAL = 0x31
V_AC_CAL = 0x32
IV_AC_CAL = 0x36
I_DC_CAL = 0x21
V_DC_CAL = 0x22
IV_DC_CAL = 0x26
I_GAIN_CAL = 0x39
V_GAIN_CAL = 0x3A

IV_GAIN_CAL = 0x3E

In [133]:
 # Conversion functions between float and fixed point

def q2f(v, q, signed=SIGNED):
    '''Signed fixed point to float conversion
    val = 24 bit integer value
    q = number of fraction bits 0..23'''
    sign = v & 0x800000
    if signed:
        assert(q <= 23)
        v = v & 0x7FFFFF
        v = v - sign
    else:
        assert(q <= 24)
    return v / (1 << q)


def f2q(val, q, signed=SIGNED):
    '''Float to fixed point. Works for signed and unsigned nubmers'''
    return int(0xFFFFFF & int(val * (1 << q)))

if False:
    # Quick eyeball test
    tv = (1, 0x10000, 0x7FFFFF, 0x800000, 0x810000, 0xFFFFFF)
    for v in tv:
        s = q2f(v, 23, True)
        u = q2f(v, 24, False)
        print(v, s, u, f2q(s, 23, True), f2q(u, 24, False))

In [123]:
class cs5490(object):
    def __init__(self, serial, debug=False):
        self.current_page = None
        self.serial = serial
        self.debug = debug
        
    def read_register(self, register):
        page = (register & 0x3F00) >> 8
        addr = register & 0x3F
        if self.current_page != page:
            self.serial.write(bytearray([CMD_PAGE | page]))
            self.current_page = page
        self.serial.write(bytearray([addr]))
        # time.sleep(0.1)
        buffer = bytearray(4)
        rb = self.serial.read(3)
        buffer[0:3] = rb
        val = struct.unpack('<L',buffer)[0]
        if self.debug:
            info = reginfo.get(register, ('unknown', 0, False, 0))
            sv = q2f(val, info[1], info[2])
            print('Read %10s %02d:%02d = 0x%06x %d %f default = 0x%06x' % 
                  (info[0], page, addr, val, val, sv, info[3]))
        return val

    def write_register(self, register, val):
        page = (register & 0x3F00) >> 8
        addr = register & 0x3F
        if self.current_page != page:
            self.serial.write(bytearray([CMD_PAGE | page]))
            self.current_page = page
        self.serial.write(bytearray([CMD_WRITE | addr]))
        buffer = struct.pack('<L', val)
        self.serial.write(buffer[0:3])    
        if self.debug:
            info = reginfo.get(register, ('unknown', 0, False, 0))
            sv = q2f(val, info[1], info[2])
            print('Write %02d:%02d = 0x%06x %d %s %s' % (page, addr, val, val, sv, info[0]))
        
    def instruction(self, instr):
        self.serial.write(bytearray([CMD_INST | instr]))
        if self.debug:
            print('Instruction %x' % instr)

In [124]:
portname = '/dev/serial/by-id/usb-FTDI_FT232R_USB_UART_A6007wZa-if00-port0'
sp = serial.Serial(port=portname, baudrate=600)
time.sleep(1)
chip = cs5490(sp, debug=True)

# Perform Soft RESET

In [125]:
chip.instruction(SOFT_RESET)

Instruction 1


# Read Hardware register Config0

In [126]:
chip.debug = True
for addr, info in reginfo.items():
    v = chip.read_register(addr)

Read    config0 00:00 = 0xc02000 12591104 12591104.000000 default = 0xc02000
Read    config1 00:01 = 0x00eeee 61166 61166.000000 default = 0x00eeee
Read serialctrl 00:07 = 0x02004d 131149 131149.000000 default = 0x02004d
Read    status0 00:23 = 0x800000 8388608 8388608.000000 default = 0x800000
Read    status1 00:24 = 0x801800 8394752 8394752.000000 default = 0x801800
Read    status2 00:25 = 0x000000 0 0.000000 default = 0x000000
Read     v_peak 00:36 = 0x000000 0 0.000000 default = 0x000000
Read     i_peak 00:37 = 0x000000 0 0.000000 default = 0x000000
Read    config2 16:00 = 0x100200 1049088 1049088.000000 default = 0x100200
Read     regchk 16:01 = 0x000000 0 0.000000 default = 0x000000
Read     v_inst 16:02 = 0x000000 0 0.000000 default = 0x000000
Read     i_inst 16:03 = 0x000000 0 0.000000 default = 0x000000
Read     p_inst 16:04 = 0x000000 0 0.000000 default = 0x000000
Read      p_avg 16:05 = 0x000000 0 0.000000 default = 0x000000
Read      i_rms 16:06 = 0x000000 0 0.000000 defaul

# Change Voltage Gain

In [127]:
gain = 0.8
gain_q = f2q(gain, 22, False)
chip.write_register(V_GAIN, gain_q)

Write 16:35 = 0x333333 3355443 0.7999999523162842 v_gain


In [128]:
v = chip.read_register(V_GAIN)
print(q2f(v, 22, False))

Read     v_gain 16:35 = 0x333333 3355443 0.800000 default = 0x400000
0.7999999523162842


# Read Instantaneous voltage,current and power

In [130]:
chip.instruction(CONT_CONV)

Instruction 15


In [132]:
chip.debug = False
for _ in range(10):
    v = chip.read_register(V_RMS)
    i = chip.read_register(I_RMS)
    p = chip.read_register(P_AVG)
    vv = q2f(v, 24, False)
    ii = q2f(i, 24, False)
    pp = q2f(p, 23)
    print("V={:5.3} I={:5.3}, P={:5.3}".format(vv, ii, pp))
    time.sleep(0.5)

V=0.169 I=0.397, P=0.067
V=0.169 I=0.397, P=0.067
V=0.169 I=0.397, P=0.067
V=0.169 I=0.397, P=0.0672
V=0.169 I=0.396, P=0.0669
V=0.169 I=0.397, P=0.0669
V=0.169 I=0.397, P=0.0669
V=0.169 I=0.397, P=0.0669
V=0.169 I=0.396, P=0.0669
V=0.169 I=0.396, P=0.0668


In [60]:
reg = TEMP
name, q, sign, default = reginfo[TEMP]
t = chip.read_register(TEMP)
print(name, q2f(t, q, sign))

temp 24.58984375
